In [1]:
import requests, re, os, time
import pandas as pd

In [2]:
keywords = '''queueing AND service interruption'''
APIkey = ''

In [3]:
current_time = time.asctime()
current_time = current_time.replace("  ","_")
current_time = current_time.replace(" ","_")
current_time = current_time.replace(":","_")

In [4]:
os.popen("mkdir "+current_time)

In [5]:
def extract_metadata(keywords,APIkey):
    textdata =pd.DataFrame()
    start = 1
    count =100
    idx = 1
    while True:
        url = 'http://api.elsevier.com/content/search/index:SCIDIR?query='+keywords+'&count='+str(count)+"&start="+str(start)
        temp_df,num_results = extract_article_info(url,start,APIkey)
        textdata=textdata.append(temp_df,ignore_index=True)
        if num_results<count:
            print("last page")
            break
        else:
            start = start+ 100 
            
    return textdata

In [6]:
def extract_article_info(url,start,APIkey):
    intextdata =pd.DataFrame()
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey':APIkey})
    results = resp.json()
    num_results = 0
    identifier_set = []
    
    num_results = len(results['search-results']['entry'])
    for idx in range(0,num_results):
        try:
            identifier_set.append(results['search-results']['entry'][idx]['dc:identifier'])
        except:
            pass
    print(round(100*len(identifier_set)/num_results,2),'% of results from',start,"~",start+num_results-1,'can be accessed')
    idx=0
    for identifier in identifier_set:
        try:
            try:
                resp_info = requests.get("http://api.elsevier.com/content/article/"+identifier+"?&view=FULL",
                                headers={'Accept':'application/json',
                                         'X-ELS-APIKey': APIkey})
                results_info = resp_info.json()
                _ = results_info["full-text-retrieval-response"]["coredata"]["dc:title"]
            except:
                resp_info = requests.get("http://api.elsevier.com/content/article/"+identifier,
                    headers={'Accept':'application/json',
                         'X-ELS-APIKey': APIkey})
                results_info = resp_info.json()
                _ = results_info["full-text-retrieval-response"]["coredata"]["dc:title"]



            if (idx+start)%1==0:
                print(idx+start,"papers are extracted")

            ### number of citation ###
            try:
                resp_c = requests.get("http://api.elsevier.com/content/search/index:SCOPUS?query=DOI("+identifier[4:]+")&field=citedby-count",
                        headers={'X-ELS-APIKey': APIkey})
                results_c = resp_c.json()
                cv = results_c['search-results']['entry']
                cc = cv[0]['citedby-count']
            except:
                cc = 0

            ### title ###
            try:        
                title=results_info["full-text-retrieval-response"]["coredata"]["dc:title"]
            except:
                title="not provided"

            ### date of publication ###
            try:        
                date=results_info["full-text-retrieval-response"]["coredata"]["prism:coverDate"]
            except:
                date="not provided"

            ### authors ###    
            try:
                authors=[]
                for x in range(0,len(results_info["full-text-retrieval-response"]["coredata"]['dc:creator'])):
                    authors.append(results_info["full-text-retrieval-response"]["coredata"]['dc:creator'][x]['$'])
            except:
                authors="not provided"

            ### Author provided keywords ###
            try:
                apk=[]
                for x in range(0,len(results_info["full-text-retrieval-response"]["coredata"]['dcterms:subject'])):
                    apk.append(results_info["full-text-retrieval-response"]["coredata"]['dcterms:subject'][x]['$'])
            except:
                apk="not provided"

            ### journal/book name ###
            try:        
                journal=results_info["full-text-retrieval-response"]["coredata"]['prism:publicationName']
            except:
                journal="not provided"

            ### type of publication ###
            try:        
                jtype=results_info["full-text-retrieval-response"]["coredata"]['prism:aggregationType']
            except:
                jtype="not provided"

            ### abstract ###    
            try:        
                abst=results_info["full-text-retrieval-response"]["coredata"]['dc:description']
            except:
                abst="not provided"

            # links for the paper
            try:
                links = results_info['full-text-retrieval-response']['coredata']['prism:url']
            except:
                links = "not provided"

            ### full body ###
            try:
                full_body = results_info['full-text-retrieval-response']['originalText'].split(results_info['full-text-retrieval-response']['coredata']['dc:description'])[-1]
            except:
                full_body = "not provided"
            f = open(current_time+'/'+str(idx+start)+'.txt','w')
            f.write(full_body)
            f.close()


            intextdata=intextdata.append({'DOI':identifier,'title':title,'date':date,'authors':authors,
                    'keywords':apk,'journal_or_book_name':journal,
                    'journal_type':jtype,'abstract':abst,"citation_count":int(cc),"paper_id":int(idx+start),"link":links},ignore_index=True)
            idx+=1
        except:
            print(idx,": full review failed")
    
    return intextdata,num_results

In [7]:
textdata = extract_metadata(keywords,APIkey)

100.0 % of results from 1 ~ 100 can be accessed
1 papers are extracted
2 papers are extracted
3 papers are extracted
4 papers are extracted
5 papers are extracted
6 papers are extracted
7 papers are extracted
8 papers are extracted
9 papers are extracted
10 papers are extracted


In [8]:
textdata.sort_values(['citation_count'],ascending=0)[:5]

,DOI,abstract,authors,citation_count,date,journal_or_book_name,journal_type,keywords,link,paper_id,title
4,DOI:10.1016/j.camwa.2009.06.021,Abstract This paper deals with the steady-stat...,"[Choudhury, Gautam, Tadj, Lotfi, Deka, Kandarpa]",31.0,2010-01-31,Computers & Mathematics with Applications,Journal,"[Stationary distribution, Random breakdown, Se...",http://api.elsevier.com/content/article/pii/S0...,5.0,A batch arrival retrial queueing system with t...
3,DOI:10.1016/j.ejor.2007.05.010,\n Abstract\n \n ...,"[Fiems, Dieter, Maertens, Tom, Bruneel, Herwig]",30.0,2008-08-01,European Journal of Operational Research,Journal,"[Queueing theory, Server interruptions, Server...",http://api.elsevier.com/content/article/pii/S0...,4.0,Queueing systems with different types of serve...
6,DOI:10.1016/S0305-0548(01)00086-7,\n Abstract\n \n ...,"[Fiems, Dieter, Steyaert, Bart, Bruneel, Herwig]",23.0,2003-01-31,Computers & Operations Research,Journal,"[Discrete-time queueing models, Service strate...",http://api.elsevier.com/content/article/pii/S0...,7.0,Analysis of a discrete-time GI–G–1 queueing mo...
0,DOI:10.1016/j.asej.2014.02.003,Abstract This paper deals with the steady stat...,"[Rajadurai, P., Saravanarajan, M.C., Chandrase...",14.0,2014-09-30,Ain Shams Engineering Journal,Journal,"[Two types of service, Re-service, Modified va...",http://api.elsevier.com/content/article/pii/S2...,1.0,"Analysis of an M[X]/(G1,G2)/1 retrial queueing..."
2,DOI:10.1016/j.peva.2011.07.002,\n Abstract\n \n ...,"[Morozov, Evsey, Fiems, Dieter, Bruneel, Herwig]",12.0,2011-12-31,Performance Evaluation,Journal,"[Queueing theory, Server interruptions, Stabil...",http://api.elsevier.com/content/article/pii/S0...,3.0,Stability analysis of multiserver discrete-tim...
